# Background Subtraction: Algorithm Validation and Verification

To run this notebook, `build.sh` the image_tools project. Then: 

    pip install jupyter matplotlib scipy scikit-image
    jupyter notebook

In [ ]:
%matplotlib inline

import sys
from os.path import abspath, join, dirname, exists

this_folder = abspath(".")
src_gen_path = abspath(join(this_folder, "src-gen"))
sys.path.append(src_gen_path)


## Python Reference Implementation

Here, we implement the **Reference Implementation**. For this we derive from the **generated abstract algorithm**. The parameters are completly handled by the generated code. We just have to elaborate on the algorithmic details here. Note: In Python we always compute the real median. The `type` parameter is ignored here (there is no `HISTOGRAMBASED_MEDIAN_APPROX` implementation)...

In [ ]:
from my_image_lib.GrayImage import GrayImage
from my_image_lib.background_subtraction.AlgoBackgroundSubtraction import AlgoBackgroundSubtraction
from my_image_lib.background_subtraction.BackgroundSubtractionResults import BackgroundSubtractionResults
from scipy.ndimage import median_filter
import numpy as np
from pylab import *
from skimage.io import imread

class MyAlgoBackgroundSubtraction(AlgoBackgroundSubtraction):
    def compute(self, inp, the_res):
        the_res.threshold.pixel = array(median_filter(inp.pixel, self.params.n), dtype=float) + self.params.threshold
        the_res.result.pixel = array(inp.pixel > the_res.threshold.pixel, dtype=float)
        the_res.threshold.w = inp.pixel.shape[1]
        the_res.threshold.h = inp.pixel.shape[0]
        the_res.result.w = inp.pixel.shape[1]
        the_res.result.h = inp.pixel.shape[0]

Now, we apply the **Reference Implementation** on an **example image** (with default parameters)...

In [ ]:
inp = GrayImage()
inp.pixel = imread("zeitung.png", as_gray=True)/255.0
inp.w = inp.pixel.shape[1]
inp.h = inp.pixel.shape[0]
ref_res = BackgroundSubtractionResults()
algo = MyAlgoBackgroundSubtraction()
algo.compute(inp, ref_res)
print(f"used parameters:\n{algo.params}")

--- and show the results:

In [ ]:
figure(figsize=[15,5])

subplot(1,3,1)
title("input")
imshow(inp.pixel, cm.gray)

subplot(1,3,2)
title("threshold (Python, reference)")
imshow(ref_res.threshold.pixel, cm.gray)

subplot(1,3,3)
title("result (Python, reference)")
imshow(ref_res.result.pixel, cm.gray);

## Run and evaluate the C++ Implementation

Now we **instantiate the C++ Algorithm** (compiled and made accessible to Python via SWIG) and **apply the C++ Algorithm** on the same data as above:

In [ ]:
from mdsd.item.io import copy
from my_image_lib_swig import GrayImage as SwigGrayImage
from my_image_lib_swig import BackgroundSubtractionResults as SwigBackgroundSubtractionResults
from my_image_lib_swig import AlgoBackgroundSubtraction as SwigAlgoBackgroundSubtraction
swig_inp = SwigGrayImage()
swig_res = SwigBackgroundSubtractionResults()
res = BackgroundSubtractionResults()

copy(inp, swig_inp)  # copy data from Python to C++

swig_algo = SwigAlgoBackgroundSubtraction.create()
swig_algo.compute(swig_inp, swig_res)

copy(swig_res, res);  # copy data back from C++ to Python

**Display the results**:

In [ ]:
figure(figsize=[15,5])

subplot(1,3,1)
title("input")
imshow(inp.pixel, cm.gray)

subplot(1,3,2)
title("threshold (C++)")
imshow(res.threshold.pixel, cm.gray)

subplot(1,3,3)
title("result (C++)")
imshow(res.result.pixel, cm.gray);

### Compare C++ and Reference Implementation

You can **see the difference of the implementations**, especially in the border region:

In [ ]:
figure(figsize=[15,7])
y = 200

subplot(2,3,1)
title("threshold (Python)")
imshow(ref_res.threshold.pixel, cm.gray)
plot([0,ref_res.threshold.w],[y,y], 'b-')

subplot(2,3,4)
title("threshold (C++)")
imshow(res.threshold.pixel, cm.gray)
plot([0,ref_res.threshold.w],[y,y], 'r--')

subplot(1,2,2)
plot(ref_res.threshold.pixel[y,:], 'b-', label="Python, reference")
plot(res.threshold.pixel[y,:], 'r--', label="C++")
legend();

border = algo.params.n//2+1
error = abs(res.threshold.pixel-ref_res.threshold.pixel)
error = error[border:-border, border:-border]
subplot(2,6,3)
boxplot(error.flatten())
title(f"errors excluding\nthe border ({border})")
subplot(2,6,9)
title(f"error locations, excluding\nthe border ({border})\n(darker = larger error)")
imshow(-error**0.1, cm.gray);